<a href="https://colab.research.google.com/github/jchrostowski/Data_Science/blob/main/Czesc_praktyczna_Chrostowski_Jan_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [196]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
# Baza filmów Movielens

path = "/content/drive/MyDrive/MAGISTERKA/"

ratings = pd.read_csv(path+'ratings.csv')
movies = pd.read_csv(path+'movies.csv')
tags = pd.read_csv(path+'tags.csv')

In [ ]:
movies.head()
movies[movies.title == 'Forrest Gump (1994)']

In [ ]:
tags.head()

In [ ]:
ratings.head()

In [ ]:
ratings.drop('timestamp', axis=1, inplace=True)
ratings.head()

In [60]:
# Filtrujemy tylko filmy, które oceniło przynajmnniej x użytkowników
x = 50
agg = ratings.groupby('movieId').count()
movie_list = agg[agg.rating >= x]
movie_list = movie_list.index.to_list()

movies = movies[movies.movieId.isin(movie_list)]


In [61]:
movies['genres'] = movies['genres'].str.replace('|',' ')
movies.head()
movies[movies.title == 'Forrest Gump (1994)']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movieId,title,genres
314,356,Forrest Gump (1994),Comedy Drama Romance War


In [62]:
movies[['title_clean', 'year']] = movies['title'].str.rsplit('(', n=1, expand=True)
movies['year'] = movies['year'].astype(str).str[:4]
movies

movies.groupby('year').count().sort_values(by='title_clean', ascending=False).head(100)
movies[movies.title == 'Forrest Gump (1994)']

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movieId,title,genres,title_clean,year
314,356,Forrest Gump (1994),Comedy Drama Romance War,Forrest Gump,1994


In [63]:
movies[movies.title == 'Forrest Gump (1994)']

,movieId,title,genres,title_clean,year
314,356,Forrest Gump (1994),Comedy Drama Romance War,Forrest Gump,1994


In [64]:
movies['year'] = movies['year'].replace(['None'], np.nan)
lista = movies['year'].value_counts()
movies[movies.title == 'Forrest Gump (1994)']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movieId,title,genres,title_clean,year
314,356,Forrest Gump (1994),Comedy Drama Romance War,Forrest Gump,1994


In [65]:
movies['year'] = pd.to_numeric(movies['year'], downcast='integer')

# Odrzucamy filmy, które są starsze niż lata 60 

movies = movies[movies['year'] >= 1960]
# movies['year'].value_counts().sort_values()
movies['decade'] = movies['year'].astype(str).str[2:4]
movies['decade'] = pd.to_numeric(movies['decade'])
movies['decade'] = movies['decade'].apply(lambda x: x // 10 * 10)
movies['decade'] = movies['decade'].astype(str)+'s'
movies['decade'] = movies['decade'].apply(lambda x: x.zfill(3))

movies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movieId,title,genres,title_clean,year,decade
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s
1,2,Jumanji (1995),Adventure Children Fantasy,Jumanji,1995,90s
2,3,Grumpier Old Men (1995),Comedy Romance,Grumpier Old Men,1995,90s
5,6,Heat (1995),Action Crime Thriller,Heat,1995,90s
6,7,Sabrina (1995),Comedy Romance,Sabrina,1995,90s
...,...,...,...,...,...,...
8372,109374,"Grand Budapest Hotel, The (2014)",Comedy Drama,"Grand Budapest Hotel, The",2014,10s
8376,109487,Interstellar (2014),Sci-Fi IMAX,Interstellar,2014,10s
8475,112852,Guardians of the Galaxy (2014),Action Adventure Sci-Fi,Guardians of the Galaxy,2014,10s
8569,116797,The Imitation Game (2014),Drama Thriller War,The Imitation Game,2014,10s


In [66]:
movies = movies.rename(columns={"title": 'title (year)', 'title_clean': 'title'})
movies['decade'].value_counts()

90s    193
00s    136
80s     53
70s     24
10s     18
60s     13
Name: decade, dtype: int64

In [67]:
movies[movies['title (year)'] == 'Forrest Gump (1994)']

,movieId,title (year),genres,title,year,decade
314,356,Forrest Gump (1994),Comedy Drama Romance War,Forrest Gump,1994,90s


In [68]:
# Ogarniczenie tylko do filmów, które oceniło 100 użytkowników 

In [69]:
# Ograniczenie ocen, tylko do ocen użytkowników, którzy ocenili 40 lub więcej filmów

rating_criteria = 40

ratings_f = ratings.groupby('userId').filter(lambda x: len(x) >= rating_criteria)
ratings_f

movie_list_rating = ratings_f.movieId.unique().tolist()


# Ile % użytkowników oceniło przynajmniej tyle tytułów? 
len(ratings_f.userId.unique()) / len(ratings.userId.unique()) * 100

70.16393442622952

In [70]:
movies = movies[movies.movieId.isin(movie_list_rating)]
movies.head()
movies.shape

movies[movies['title (year)'] == 'Forrest Gump (1994)']

,movieId,title (year),genres,title,year,decade
314,356,Forrest Gump (1994),Comedy Drama Romance War,Forrest Gump,1994,90s


In [71]:
movies

,movieId,title (year),genres,title,year,decade
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s
1,2,Jumanji (1995),Adventure Children Fantasy,Jumanji,1995,90s
2,3,Grumpier Old Men (1995),Comedy Romance,Grumpier Old Men,1995,90s
5,6,Heat (1995),Action Crime Thriller,Heat,1995,90s
6,7,Sabrina (1995),Comedy Romance,Sabrina,1995,90s
...,...,...,...,...,...,...
8372,109374,"Grand Budapest Hotel, The (2014)",Comedy Drama,"Grand Budapest Hotel, The",2014,10s
8376,109487,Interstellar (2014),Sci-Fi IMAX,Interstellar,2014,10s
8475,112852,Guardians of the Galaxy (2014),Action Adventure Sci-Fi,Guardians of the Galaxy,2014,10s
8569,116797,The Imitation Game (2014),Drama Thriller War,The Imitation Game,2014,10s


In [72]:
movies = movies.set_index('movieId')
movies

,title (year),genres,title,year,decade
movieId,,,,,
1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s
2,Jumanji (1995),Adventure Children Fantasy,Jumanji,1995,90s
3,Grumpier Old Men (1995),Comedy Romance,Grumpier Old Men,1995,90s
6,Heat (1995),Action Crime Thriller,Heat,1995,90s
7,Sabrina (1995),Comedy Romance,Sabrina,1995,90s
...,...,...,...,...,...
109374,"Grand Budapest Hotel, The (2014)",Comedy Drama,"Grand Budapest Hotel, The",2014,10s
109487,Interstellar (2014),Sci-Fi IMAX,Interstellar,2014,10s
112852,Guardians of the Galaxy (2014),Action Adventure Sci-Fi,Guardians of the Galaxy,2014,10s


In [73]:
top100 = ratings_f.groupby('movieId')['rating'].count().to_frame(name='ratings').sort_values(by='ratings', ascending=False).head(100)
top100 = top100.join(movies['title (year)'])
top100 = top100.iloc[:, [1,0]]
top100 = top100.drop('ratings', axis=1)
top100['rating'] = np.nan
top100.to_excel(path+'Top 100 Movies.xlsx')
top100

,title (year),rating
movieId,,
356,Forrest Gump (1994),NaN
296,Pulp Fiction (1994),NaN
318,"Shawshank Redemption, The (1994)",NaN
2571,"Matrix, The (1999)",NaN
593,"Silence of the Lambs, The (1991)",NaN
...,...,...
33794,Batman Begins (2005),NaN
587,Ghost (1990),NaN
2,Jumanji (1995),NaN


In [74]:
# Filmy ocenione przez użytkownika
rated_movies = pd.read_excel(path+'Rated Movies.xlsx')
rated_movies = rated_movies[['movieId','rating']]
rated_movies = rated_movies[rated_movies.rating.notna()]
rated_movies

,movieId,rating
0,356,4.0
1,296,4.0
2,318,4.0
3,2571,1.0
4,593,4.0
9,2959,3.0
11,2858,4.0
12,1,4.0
13,47,4.0
19,4993,4.0


In [75]:
# Dodanie ocen użytkownika do bazy
user_id = ratings_f.userId.max()+1
rated_movies['userId'] = user_id
rated_movies = rated_movies[rated_movies.movieId.isin(movie_list_rating)] # do sprawdzenia
rated_movies

,movieId,rating,userId
0,356,4.0,611
1,296,4.0,611
2,318,4.0,611
3,2571,1.0,611
4,593,4.0,611
9,2959,3.0,611
11,2858,4.0,611
12,1,4.0,611
13,47,4.0,611
19,4993,4.0,611


In [76]:
ratings_f = ratings_f.append(rated_movies)
ratings_f

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
77,611,10,3.0
80,611,8961,4.0
86,611,5349,4.0
91,611,586,2.0


In [77]:
# oceny = pd.merge(ratings_f, movies, how="left")
# oceny = oceny[['userId','movieId','title','rating']]
# oceny

In [78]:
# Łączenie zbioru movies i tags

mixed = pd.merge(movies, tags, on = "movieId", how='left')
mixed.head()

,movieId,title (year),genres,title,year,decade,userId,tag,timestamp
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s,336.0,pixar,1.139046e+09
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s,474.0,pixar,1.137207e+09
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,Toy Story,1995,90s,567.0,fun,1.525286e+09
3,2,Jumanji (1995),Adventure Children Fantasy,Jumanji,1995,90s,62.0,fantasy,1.528844e+09
4,2,Jumanji (1995),Adventure Children Fantasy,Jumanji,1995,90s,62.0,magic board game,1.528844e+09


In [79]:
mixed.fillna("", inplace=True)
mixed = pd.DataFrame(mixed.groupby("movieId")['tag'].apply(lambda x: " ".join(x)))

Final = pd.merge(movies, mixed, on = "movieId", how="left")
Final['metadata'] = Final[['decade','genres','tag']].apply(lambda x: " ".join(x), axis = 1)
Final = Final.drop(['title (year)', 'year', 'decade'], axis=1)
Final['movieId'] = Final.index
Final.head()


,genres,title,tag,metadata,movieId
movieId,,,,,
1,Adventure Animation Children Comedy Fantasy,Toy Story,pixar pixar fun,90s Adventure Animation Children Comedy Fantas...,1
2,Adventure Children Fantasy,Jumanji,fantasy magic board game Robin Williams game,90s Adventure Children Fantasy fantasy magic b...,2
3,Comedy Romance,Grumpier Old Men,moldy old,90s Comedy Romance moldy old,3
6,Action Crime Thriller,Heat,,90s Action Crime Thriller,6
7,Comedy Romance,Sabrina,remake,90s Comedy Romance remake,7


In [80]:
Final['metadata']

movieId
1         90s Adventure Animation Children Comedy Fantas...
2         90s Adventure Children Fantasy fantasy magic b...
3                              90s Comedy Romance moldy old
6                                90s Action Crime Thriller 
7                                 90s Comedy Romance remake
                                ...                        
109374                                    10s Comedy Drama 
109487    10s Sci-Fi IMAX black hole sci-fi time-travel ...
112852    10s Action Adventure Sci-Fi funny Great Visual...
116797     10s Drama Thriller War inspirational intelligent
122904                  10s Action Adventure Comedy Sci-Fi 
Name: metadata, Length: 437, dtype: object

In [81]:
# !pip uninstall scikit-learn
# !pip install scikit-learn==0.13

# Stworzenie macierzy filmy x tagi
# Im wieksza liczba tym "wazniejsze" slowo 

# jedna z metod obliczania wagi słów w oparciu o liczbę ich wystąpień, należąca do grupy algorytmów obliczających statystyczne wagi termów

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(Final['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=Final.index.tolist())
print(tfidf_df.shape)
tfidf_df.head()

(437, 834)


,0,1,2,3,4,5,6,7,8,9,...,824,825,826,827,828,829,830,831,832,833
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.129117,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094806,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.185649,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.413388,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266599,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
tfidf_df

,0,1,2,3,4,5,6,7,8,9,...,824,825,826,827,828,829,830,831,832,833
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.129117,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.094806,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.185649,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.413388,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266599,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109374,0.0,0.834486,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109487,0.0,0.158418,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112852,0.0,0.269766,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116797,0.0,0.415779,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# !pip uninstall scikit-learn
# !pip install scikit-learn==0.9
# !pip install sklearn
# print(sklearn.__version__)

import sklearn
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200)
latent_matrix = svd.fit_transform(tfidf_df)

latent_matrix.shape

# SVD - Rozkład według wartości osobliwych - redukcja wymiaru macierzy



(437, 200)

In [84]:
Final

,genres,title,tag,metadata,movieId
movieId,,,,,
1,Adventure Animation Children Comedy Fantasy,Toy Story,pixar pixar fun,90s Adventure Animation Children Comedy Fantas...,1
2,Adventure Children Fantasy,Jumanji,fantasy magic board game Robin Williams game,90s Adventure Children Fantasy fantasy magic b...,2
3,Comedy Romance,Grumpier Old Men,moldy old,90s Comedy Romance moldy old,3
6,Action Crime Thriller,Heat,,90s Action Crime Thriller,6
7,Comedy Romance,Sabrina,remake,90s Comedy Romance remake,7
...,...,...,...,...,...
109374,Comedy Drama,"Grand Budapest Hotel, The",,10s Comedy Drama,109374
109487,Sci-Fi IMAX,Interstellar,black hole sci-fi time-travel Christopher Nola...,10s Sci-Fi IMAX black hole sci-fi time-travel ...,109487
112852,Action Adventure Sci-Fi,Guardians of the Galaxy,funny Great Visuals humorous unlikely hero,10s Action Adventure Sci-Fi funny Great Visual...,112852


In [85]:
Final.index

Int64Index([     1,      2,      3,      6,      7,     10,     11,     16,
                17,     19,
            ...
             91500,  91529,  96079,  99114, 106782, 109374, 109487, 112852,
            116797, 122904],
           dtype='int64', name='movieId', length=437)

In [86]:
# macierz movieId x metadane (zfaktoryzowane)

n = 200
latent_matrix_1_df = pd.DataFrame(latent_matrix[:,0:n], index=Final.movieId.tolist())
latent_matrix_1_df



,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,0.220602,0.142263,-0.199544,0.146805,-0.233932,-0.070905,0.172307,-0.203435,0.051094,-0.069767,...,0.008153,0.010861,0.022419,0.002322,0.016467,0.024391,0.006998,-0.022739,0.013950,0.001288
2,0.137835,0.055655,-0.116215,0.114854,-0.176538,-0.150147,0.141960,-0.017433,-0.082373,0.100843,...,0.003413,0.001657,-0.004259,-0.001977,0.000865,-0.004406,0.003163,-0.003339,-0.000883,0.012986
3,0.196813,0.244870,-0.085115,-0.083968,0.059815,-0.000267,-0.110219,0.034739,0.128889,0.045483,...,-0.006386,0.001207,-0.000254,0.002943,0.001385,-0.000392,-0.004283,-0.006322,-0.000057,0.006920
6,0.611053,-0.138264,0.493340,-0.367543,-0.315202,0.109440,0.112175,0.041804,0.189055,-0.044472,...,0.017499,-0.004944,-0.006199,0.001399,0.012310,-0.003277,0.002448,0.005304,0.001702,-0.000671
7,0.284132,0.354309,-0.124237,-0.122852,0.088573,0.000207,-0.161766,0.049668,0.191996,0.070594,...,-0.000909,0.002446,-0.004012,-0.000570,0.000821,-0.000404,0.005476,0.000394,-0.001665,-0.003334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109374,0.260048,0.260417,0.002620,0.047808,0.186169,-0.019331,0.128195,-0.057685,-0.248432,-0.087233,...,0.005307,-0.012159,-0.000919,0.003789,0.005814,-0.026925,-0.012167,0.006705,0.021970,-0.000782
109487,0.175070,-0.270323,-0.193627,0.019953,0.254297,0.060354,0.061067,-0.109361,-0.030262,0.030114,...,-0.007716,0.000770,0.022787,0.030736,-0.005049,-0.033814,0.021756,-0.023065,0.004903,-0.012835
112852,0.185612,-0.201254,-0.119800,0.029079,0.050581,-0.032635,0.001864,-0.004341,-0.029514,-0.053535,...,-0.020657,-0.017304,-0.031571,-0.018982,-0.017451,0.071576,0.060295,0.005052,-0.023434,0.062308
116797,0.154126,-0.012399,0.165604,-0.014815,0.091456,-0.193876,0.104306,-0.021345,-0.107095,-0.049226,...,-0.017245,0.009413,0.091208,-0.031785,0.050838,0.083647,0.029124,0.048747,-0.025184,-0.040333


In [87]:
ratings_f

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
77,611,10,3.0
80,611,8961,4.0
86,611,5349,4.0
91,611,586,2.0


In [88]:
# CZĘŚĆ COLLABORATIVE


In [89]:
# Filtrujemy oceny tylko dla filmów, które bedą brane pod uwagę
movies = movies.reset_index()
ratings_f1 = pd.merge(movies, ratings_f, on="movieId", how="left")
ratings_f1['movieId'].value_counts()
ratings_f1 = ratings_f1[['userId','title (year)','rating']]
ratings_f1

,userId,title (year),rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5
...,...,...,...
37428,561,Deadpool (2016),2.0
37429,586,Deadpool (2016),4.0
37430,596,Deadpool (2016),4.0
37431,599,Deadpool (2016),3.5


USER-BASED COLLABORATIVE FILTERING

In [221]:

def user_based_recommendation(picked_userid, number_of_similar_users=3, number_of_recommendations=5, min_similarity=0.2):

  # picked_userid = 611
  # number_of_similar_users = 2
  # number_of_recommendations = 5
  # min_similarity = 0.2

  # Stworznie macierzy użytkownicy x filmy
  ub_matrix = ratings_f1.pivot_table(index='userId', columns='title (year)', values='rating')

  # Normalizacja macierzy
  ub_matrix_norm = ub_matrix.subtract(ub_matrix.mean(axis=1), axis='rows')

  # Macierz cosinusowego podobieństwa użytkowników
  users = ub_matrix_norm.T.columns
  cosine_matrix = cosine_similarity(ub_matrix_norm.fillna(0))
  user_similarity = pd.DataFrame(cosine_matrix, index = users, columns=users)

  # Znalezienie n podobnych użytkowników
  similar_users = user_similarity[user_similarity[picked_userid] > min_similarity][picked_userid].sort_values(ascending=False)[1:number_of_similar_users+1]

  # Usunięcie filmów już obejrzanych przez wybranego użytkownika
  picked_user_watched = ub_matrix_norm[ub_matrix_norm.index == picked_userid].dropna(axis=1, how='all')
  similar_user_movies = ub_matrix_norm[ub_matrix_norm.index.isin(similar_users.index)].dropna(axis=1, how='all')
  similar_user_movies.drop(picked_user_watched.columns, axis=1, inplace=True, errors='ignore')

  # Wybranie filmów do rekomendacji
  item_score = {}
  movie_list = similar_user_movies.columns

  for i in movie_list:
    movie_rating = similar_user_movies[i]
    total = 0
    count = 0
    for u in similar_users.index:
      if pd.isna(movie_rating[u]) == False:
        score = similar_users[u] * movie_rating[u]
        total += score
        count +=1
    item_score[i] = round(total / count, 3)
  item_score = pd.DataFrame(item_score.items(), columns=['Movie', 'score'])
  ranked_movie_score = item_score.sort_values(by='score', ascending=False).head(number_of_recommendations)
  return ranked_movie_score

user_based_recommendation(611)

,Movie,score
91,Shutter Island (2010),0.289
113,"Usual Suspects, The (1995)",0.289
95,"South Park: Bigger, Longer and Uncut (1999)",0.289
97,Star Trek: Generations (1994),0.245
78,"Piano, The (1993)",0.245


In [ ]:
# Porównywanie użytkowników 

# ub_matrix_norm[ub_matrix_norm.index.isin([362,50])].dropna(axis='columns', how='any')
# ub_matrix[ub_matrix.index.isin([362,50])].dropna(axis='columns', how='any')

ITEM-BASED COLLABORATIVE FILTERING

In [ ]:

def item_based_rec(picked_userid, number_of_similar_items=5, number_of_recommendations=10):

  # Stworzenie macierzy filmy x użytkownicy
  ib_matrix = ratings_f1.pivot_table(index='title (year)', columns = 'userId', values = 'rating')

  # Normalizacja macierzy
  ib_matrix_norm = ib_matrix.subtract(ib_matrix.mean(axis=1), axis=0) # informuje czy dany film został oceniony powyżej/poniżej średniej dla na. mean - kolumna , subtract - wiersz 

  # Macierz korelacji
  item_similarity = ib_matrix_norm.T.corr()

  # Filmy, których wybrany użytkownik nie oglądał
  picked_userid_unwatched = pd.DataFrame(ib_matrix_norm[picked_userid].isna()).reset_index()
  picked_userid_unwatched = picked_userid_unwatched[picked_userid_unwatched[picked_userid]==True]['title (year)'].values.tolist() 
  # Filmy, które wybrany użytkownik oglądał 
  picked_userid_watched = pd.DataFrame(ib_matrix_norm[picked_userid].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={picked_userid:'rating'})
  
  rating_prediction = []

  # Pętla po nieoglądanych filmach      
  for picked_movie in picked_userid_unwatched: 
    # Obliczenie korelacji filmu do niewidzianych do innych filmów
    picked_movie_similarity_score = item_similarity[[picked_movie]].reset_index().rename(columns={picked_movie:'similarity_score'})
    # Stworznie tabeli z podobieństwem filmu do oglądanych przez użytkownika filmów (i ich znormalizowanymi ocenami)
    picked_userid_watched_similarity = pd.merge(left=picked_userid_watched, 
                                                right=picked_movie_similarity_score, 
                                                on='title (year)', 
                                                how='inner')\
                                        .sort_values('similarity_score', ascending=False)[:number_of_similar_items]
    # Obliczenie wyniku (średniej ważonej) filmu 
    predicted_rating = round(np.average(picked_userid_watched_similarity['rating'], 
                                        weights=picked_userid_watched_similarity['similarity_score']), 3)
    
    # Zapisanie wyniku
    rating_prediction.append(predicted_rating)
  # Utworzenie dataframe
  recommended_movies = {'title (year)': picked_userid_unwatched, 'score': rating_prediction}
  recommended_movies = pd.DataFrame(recommended_movies)
  recommended_movies = recommended_movies.sort_values(by='score', ascending=False).head(number_of_recommendations)
  return recommended_movies

item_based_rec(611)


Poprzednia wersja

In [101]:
# # WYMAGANIA:

# # Sprawdz ktore filmy zostaly ocenione przez uzytkownika
# # stworz z tego liste movieId
# # dla kazdego movieID wygeneruj ranking 10 by hybrid
# # dodaj do jednego DataFrame
# # Usuń z niego filmy, które zostały przez niego obejrzane
# # Posrotuj wyniki
# # Wyswietl tytuły wraz z Rekomendacją w procentach

# def recommend(userId):

#   rated_movies = ratings_f[(ratings_f.userId == userId) & (ratings_f.rating >= 4)]["movieId"].tolist() # poprawic na ratings_f2
#   recommendations = []
#   # print(rated_movies)

#   for movie in rated_movies:

#     # Content
#     a_1 = np.array(latent_matrix_1_df.loc[movie]).reshape(1,-1)
#     # Collaborative
#     a_2 = np.array(latent_matrix_2_df.loc[movie]).reshape(1,-1)

#     score_1 = cosine_similarity(latent_matrix_1_df, a_1).reshape(-1)*100
#     score_2 = cosine_similarity(latent_matrix_2_df, a_2).reshape(-1)*100

#     hybrid = score_1*0.4 + score_2*0.6 # Ratio 
#     movie_title = movies.loc[movies['movieId'] == movie, 'title'].item()

#     dictDf = {'content': score_1, 'collaborative': score_2, 'score': hybrid, "based on": movie_title}
#     similar = pd.DataFrame(dictDf, index = latent_matrix_1_df.index )

#     similar.sort_values('score', ascending=False, inplace=True)
#     similar = similar[1:11] # TOP 10 wyników
#     recommendations.append(similar)
  
#   recommendations = pd.concat(recommendations, ignore_index=False)
#   recommendations.sort_values('score', ascending=False, inplace=True)
#   recommendations['movieId'] = recommendations.index
#   recommendations = recommendations[~recommendations['movieId'].isin(rated_movies)] # wyklucza ocenione
#   recommendations = recommendations.merge(movies[['movieId','title']], how='left', on='movieId')
#   recommendations = recommendations.drop_duplicates(subset=['title'])
#   recommendations = recommendations[0:19] # Pokaż 20 rekomendacji

#   return recommendations

# recommend(user_id)

NameError: ignored